In [1]:
# Imports
import pandas as pd
import numpy as np
import json
import unicodedata
import os

In [2]:
# Declaring path from files

filepath = "../Teste Técnico - Analista de dados (Brazil, São Paulo)"

ordersFile = "orders.csv"
customersFile = "customers.json"
webEventsFile = "web_events.csv"

ordersPath = os.path.join(filepath, ordersFile)
customersPath = os.path.join(filepath, customersFile)
webEventsPath = os.path.join(filepath, webEventsFile)

In [3]:
# Tables reads
orders = pd.read_csv(ordersPath, encoding= 'utf-8', 
                    sep = ',', index_col= False)

webEvents = pd.read_csv(webEventsPath, encoding= 'utf-8', 
                    sep = ',', index_col= False)

customers = pd.read_json(customersPath, encoding= 'utf-8')

In [4]:
customers.head(2)

,id,full_name,email,registration_date,birth_date,state,phone_number,customer_segment
0,1,Eduardo Mendes,eduardo.mendes@exemplo.com,2022-04-04,04-12-2002,MG,(15) 8939-9366,A
1,2,Carla Silva,carla.silva@exemplo.com,2020-04-27,17-09-1961,Rio Grande do Sul,(31) 2097-3369,D


In [24]:
webEvents.head(2)

,event_id,user_email,session_id,event_type,event_timestamp,page_url,user_agent,metadata
0,1,gustavo.cardoso@exemplo.com,e8bb52fd-4ccc-432d-8817-cec02ca4de67,logout,2024-03-23 00:21:17,/search,Mozilla/5.0 (iPhone; CPU iPhone OS 14_2 like M...,"{""page"": ""/checkout"", ""duration"": 11, ""referre..."
1,2,renata.lima@exemplo.com,e8c7d9eb-1f16-43b7-ba2c-76592c695b7a,logout,2024-04-12 22:12:23,/product/P001,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7),"{""page"": ""/search"", ""duration"": 379, ""referrer..."


In [5]:
# Normalize webEvents table transforming metadata column with format json as row : column

metadata_objects = webEvents['metadata'].apply(json.loads)

metadata_normalized = pd.json_normalize(metadata_objects)

metadata_normalized.columns = ['metadata_' + col for col in metadata_normalized.columns]

webEvents_normalized = pd.concat([webEvents.drop('metadata', axis= 1), metadata_normalized], axis= 1)

In [6]:
# Extract and create a column product_code with the code of product on page_url
webEvents_normalized['product_code'] = webEvents_normalized['page_url'].str.extract(r'(P\d+)')

In [7]:
webEvents_normalized.head(1)

,event_id,user_email,session_id,event_type,event_timestamp,page_url,user_agent,metadata_page,metadata_duration,metadata_referrer,product_code
0,1,gustavo.cardoso@exemplo.com,e8bb52fd-4ccc-432d-8817-cec02ca4de67,logout,2024-03-23 00:21:17,/search,Mozilla/5.0 (iPhone; CPU iPhone OS 14_2 like M...,/checkout,11,instagram.com,NaN


In [8]:
# Just change ',' for '-'.
orders['shipping_address'] = orders['shipping_address'].str.replace(',', ' - ')

In [9]:
# To standardize the column order_date as date
orders['order_date'] = orders['order_date'].astype('datetime64[ns]')

In [10]:
orders.head(2)

,order_id,customer_id,product_code,order_date,amount,currency,order_status,shipping_address,payment_method
0,1001,64,P002,2023-10-09,263.40,EUR,pending,Rua 105 - Bairro Fernandes - Cidade X,paypal
1,1002,15,P011,2023-05-30,2903.81,BRL,cancelled,Rua 80 - Bairro Costa - Cidade X,pix


In [11]:
# Transform amount format based on currency column
def format_amount(orders):
    value = float(orders['amount'])
    currency = orders['currency']

    if currency == "BRL":
        return f"{value:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    else:
        return f"{value:,.2f}"
    
orders['amount'] = orders.apply(format_amount, axis = 1)

In [12]:
orders.head(2)

,order_id,customer_id,product_code,order_date,amount,currency,order_status,shipping_address,payment_method
0,1001,64,P002,2023-10-09,263.40,EUR,pending,Rua 105 - Bairro Fernandes - Cidade X,paypal
1,1002,15,P011,2023-05-30,"2.903,81",BRL,cancelled,Rua 80 - Bairro Costa - Cidade X,pix


In [13]:
# Function to remove accent from email and name
def remover_acentos(texto):
    return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
customers['email'] = customers['email'].apply(remover_acentos)
customers['full_name'] = customers['full_name'].apply(remover_acentos)

In [14]:
# To standardize the column registration_date as date
customers['registration_date'] = customers['registration_date'].astype('datetime64[ns]')
customers['birth_date'] = customers['birth_date'].astype('datetime64[ns]')

In [15]:
# Standardizing the states names
customers['state'] = np.where(customers['state'] == "Rio Grande do Sul", "RS",
                     np.where(customers['state'] == "Minas Gerais", "MG",
                     np.where(customers['state'] == "São Paulo", "SP",
                     np.where(customers['state'] == "Rio de Janeiro", "RJ",
                     np.where(customers['state'] == "Bahia", "BA",
              customers['state'])))))

In [16]:
# Count emails and names duplicates
customers['countName'] = customers.groupby('full_name')['full_name'].transform('count')
customers['countEmail'] = customers.groupby('email')['email'].transform('count')

In [17]:
customers.head(2)

,id,full_name,email,registration_date,birth_date,state,phone_number,customer_segment,countName,countEmail
0,1,Eduardo Mendes,eduardo.mendes@exemplo.com,2022-04-04,2002-04-12,MG,(15) 8939-9366,A,1,1
1,2,Carla Silva,carla.silva@exemplo.com,2020-04-27,1961-09-17,RS,(31) 2097-3369,D,2,2


In [18]:
# Using lambda function to apply a empty email if counting is greater ou equal a 2

# customers['full_name'] = customers.apply(
#     lambda id: f"{id['full_name']} {id['id']}" if id['countName'] >= 2 else id['full_name'], axis = 1
# )

customers['email'] = customers.apply(
    #lambda id: id['email'].replace('@', f".{id['id']}@") if id['countEmail'] >= 2 else id['email'], axis = 1
    lambda id: '' if id['countEmail'] >= 2 else id['email'], axis=1
)

In [19]:
customers.head(2)

,id,full_name,email,registration_date,birth_date,state,phone_number,customer_segment,countName,countEmail
0,1,Eduardo Mendes,eduardo.mendes@exemplo.com,2022-04-04,2002-04-12,MG,(15) 8939-9366,A,1,1
1,2,Carla Silva,,2020-04-27,1961-09-17,RS,(31) 2097-3369,D,2,2


In [20]:
# Merge tables using the relationship and drop some unused columns
joinedTable = orders.merge(customers, left_on= 'customer_id', right_on= 'id', how= 'left').join(webEvents_normalized, lsuffix= '_orders', 
                                                                                                rsuffix= '_events', how = 'left').drop(
                                                                                                    ['id', 'user_email', 'countName', 
                                                                                                     'countEmail'], axis='columns')

In [21]:
joinedTable.columns

Index(['order_id', 'customer_id', 'product_code_orders', 'order_date',
       'amount', 'currency', 'order_status', 'shipping_address',
       'payment_method', 'full_name', 'email', 'registration_date',
       'birth_date', 'state', 'phone_number', 'customer_segment', 'event_id',
       'session_id', 'event_type', 'event_timestamp', 'page_url', 'user_agent',
       'metadata_page', 'metadata_duration', 'metadata_referrer',
       'product_code_events'],
      dtype='object')

In [22]:
joinedTable.head(2)

,order_id,customer_id,product_code_orders,order_date,amount,currency,order_status,shipping_address,payment_method,full_name,...,event_id,session_id,event_type,event_timestamp,page_url,user_agent,metadata_page,metadata_duration,metadata_referrer,product_code_events
0,1001,64,P002,2023-10-09,263.40,EUR,pending,Rua 105 - Bairro Fernandes - Cidade X,paypal,Juliana Oliveira,...,1,e8bb52fd-4ccc-432d-8817-cec02ca4de67,logout,2024-03-23 00:21:17,/search,Mozilla/5.0 (iPhone; CPU iPhone OS 14_2 like M...,/checkout,11,instagram.com,NaN
1,1002,15,P011,2023-05-30,"2.903,81",BRL,cancelled,Rua 80 - Bairro Costa - Cidade X,pix,Renata Rocha,...,2,e8c7d9eb-1f16-43b7-ba2c-76592c695b7a,logout,2024-04-12 22:12:23,/product/P001,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7),/search,379,facebook.com,P001


In [23]:
# Save to csv file
joinedTable.to_csv('customersData.csv', index= False, encoding= 'utf8')